# Physical Object: Pipe

The `Pipe` class models the flow of water through a pipe, connecting two points in a water system. The model uses the Darcy-Weisbach equation to calculate flow based on the pressure difference (head) between its ends.

The model can operate in two modes:
1. **Gravity Flow**: If no inflow is specified, the flow is calculated based on the `upstream_head` and `downstream_head` provided in the `action`.
2. **Forced Flow**: If an inflow is set (e.g., by an upstream pump), that inflow is treated as the pipe's outflow, and the model instead calculates the resulting head loss.

### State Variables
- `outflow` (float): The calculated flow rate through the pipe (m^3/s).
- `head_loss` (float): The loss of pressure head due to friction in the pipe (m).

### Parameters
- `length` (float): The length of the pipe (m).
- `diameter` (float): The diameter of the pipe (m).
- `friction_factor` (float): The Darcy friction factor for the pipe (dimensionless).

## Simulation Example (Gravity Flow)

The following example simulates a pipe connecting two points with a varying upstream head. This demonstrates how the outflow changes in response to the changing pressure differential.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from swp.simulation_identification.physical_objects.pipe import Pipe
from swp.core.interfaces import State, Parameters

# 1. Define initial state and parameters
initial_state = State(outflow=0.0, head_loss=0.0)
parameters = Parameters(
    length=500.0,
    diameter=0.5,
    friction_factor=0.02
)

# 2. Create a Pipe instance
pipe = Pipe(name="main_pipe", initial_state=initial_state, parameters=parameters)

# 3. Simulation settings
dt = 1.0
simulation_duration = 100
num_steps = int(simulation_duration / dt)
history = []
upstream_heads = []

# 4. Run the simulation loop
downstream_head = 50.0  # Constant downstream head
for t in range(num_steps):
    # Vary the upstream head over time using a sine wave
    upstream_head = 55.0 + 5.0 * np.sin(2 * np.pi * t / simulation_duration)
    upstream_heads.append(upstream_head)
    
    action = {
        'upstream_head': upstream_head,
        'downstream_head': downstream_head
    }
    
    current_state = pipe.step(action=action, dt=dt)
    history.append(current_state.copy())

print("Pipe simulation complete.")

## Results and Visualization

The plots show the direct relationship between the head difference across the pipe and the resulting outflow.

In [ ]:
# Extract data from history
time = [i * dt for i in range(num_steps)]
outflows = [h['outflow'] for h in history]
head_losses = [h['head_loss'] for h in history]

# Create a DataFrame
df = pd.DataFrame({
    'Time': time,
    'Upstream Head': upstream_heads,
    'Outflow': outflows,
    'Head Loss': head_losses
})

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Heads
ax1.plot(df['Time'], df['Upstream Head'], label='Upstream Head', color='blue')
ax1.axhline(y=downstream_head, color='red', linestyle='--', label='Downstream Head')
ax1.set_ylabel('Head (m)')
ax1.set_title('Pipe Flow Simulation')
ax1.grid(True)
ax1.legend()

# Outflow
ax2.plot(df['Time'], df['Outflow'], label='Outflow', color='green')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Outflow (m^3/s)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()